In [43]:
import logging
import pandas as pd
import time

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

logging.basicConfig(level=logging.INFO)

In [36]:
edge_options = Options()
user_agent = "Edge/122.0.2365.92"
edge_options.add_argument(f'user-agent={user_agent}')
driver = webdriver.Edge(options=edge_options)
driver.get("https://www.zillow.com/homes/fsbo/?searchQueryState=%7B%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22north%22%3A42.78759937330219%2C%22south%22%3A41.82788354480652%2C%22east%22%3A-70.5237973397179%2C%22west%22%3A-72.13054782799915%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22schu%22%3A%7B%22value%22%3Afalse%7D%2C%22sche%22%3A%7B%22value%22%3Afalse%7D%2C%22schm%22%3A%7B%22value%22%3Afalse%7D%2C%22schh%22%3A%7B%22value%22%3Afalse%7D%2C%22schp%22%3A%7B%22value%22%3Afalse%7D%2C%22schr%22%3A%7B%22value%22%3Afalse%7D%2C%22schc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22category%22%3A%22cat2%22%2C%22customRegionId%22%3A%22b522066044X1-CR1rt49s8urjnaj_15plh3%22%2C%22pagination%22%3A%7B%7D%7D")
time.sleep(3)

for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
html_content = driver.page_source
driver.quit()

soup = BeautifulSoup(html_content, 'html.parser')

# List of tags to remove
unwanted_tags = ['script', 'style', 'button', 'input', 'img', 'video', 'head', 'svg']

for tag in unwanted_tags:
    for unwanted_tag in soup.find_all(tag):
        unwanted_tag.decompose()  # Remove the tag from the soup


In [ ]:
# Find all 'li' tags within the 'ul' tag
listings = soup.find('ul', class_='StyledSearchListWrapper-srp-8-100-4__sc-1ieen0c-0').find_all('li', recursive=False)

# Initialize a list to hold all the extracted information
extracted_info = []

for listing in listings:
    # For each 'li', find the 'article' tag with 'data-test' attribute 'property-card'
    article = listing.find('article', {'data-test': 'property-card'})
    
    # Proceed if the article tag exists
    if article:
        # Extract the desired information
        price = article.find('span', {'data-test': 'property-card-price'}).get_text(strip=True) if article.find('span', {'data-test': 'property-card-price'}) else 'No Price'
        address = article.find('address', {'data-test': 'property-card-addr'}).get_text(strip=True) if article.find('address', {'data-test': 'property-card-addr'}) else 'No Address'
        details_list = article.find('ul', class_='StyledPropertyCardHomeDetailsList-c11n-8-100-4__sc-1j0som5-0')
        
        # Extract beds, baths, and sqft separately
        beds = baths = sqft = 'N/A'  # Default values
        if details_list:
            details = details_list.find_all('li')
            if len(details) > 0:
                beds = details[0].get_text(strip=True)
            if len(details) > 1:
                baths = details[1].get_text(strip=True)
            if len(details) > 2:
                sqft = details[2].get_text(strip=True)
        
        extracted_info.append({
            'price': price, 
            'address': address, 
            'beds': beds,
            'baths': baths,
            'sqft': sqft
        })

    else:
        logging.info('Article tag not found')
        print('Rerun the driver')

In [46]:
# Convert the extracted information to a pandas DataFrame
df = pd.DataFrame(extracted_info)

# Save the DataFrame to a CSV file
csv_file_path = 'zillow_listings.csv'
df.to_csv(csv_file_path, index=False)
print(f'Saved extracted information to {csv_file_path}')

# Alternatively, to save it as an Excel file, you can do:
excel_file_path = 'zillow_listings.xlsx'
df.to_excel(excel_file_path, index=False)
print(f'Saved extracted information to {excel_file_path}')

Saved extracted information to zillow_listings.csv
Saved extracted information to zillow_listings.xlsx
